In [21]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [22]:
# Importing the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [23]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
# Choosing the necessary variables for Association using Apriori
import numpy as np
titanic = train[['Pclass','Sex','Age','Survived']]
titanic.head()

,Pclass,Sex,Age,Survived
0,3,male,22.0,0
1,1,female,38.0,1
2,3,female,26.0,1
3,1,female,35.0,1
4,3,male,35.0,0


In [25]:
# Filling missing values for Age
titanic["Age"].fillna(titanic.groupby("Pclass")["Age"].transform("median"), inplace=True)
titanic.isnull().sum()

C:\Users\sudhe\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Pclass      0
Sex         0
Age         0
Survived    0
dtype: int64

In [26]:
# Categorizing Age
a = []
for i in titanic['Age']:
  if i <= 15:
    a.append('Child')
  else:
    a.append('Adult')
titanic['Age_cat'] = a
titanic= titanic.drop('Age', axis=1)
titanic.head()

C:\Users\sudhe\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Pclass,Sex,Survived,Age_cat
0,3,male,0,Adult
1,1,female,1,Adult
2,3,female,1,Adult
3,1,female,1,Adult
4,3,male,0,Adult


In [27]:
# Categorizing Survived
x = []
for i in titanic['Survived']:
  if i == 0:
    x.append('Dead')
  else:
    x.append('Survived')
titanic['Survived_Cat'] = x
titanic= titanic.drop('Survived', axis=1)
titanic.head()

,Pclass,Sex,Age_cat,Survived_Cat
0,3,male,Adult,Dead
1,1,female,Adult,Survived
2,3,female,Adult,Survived
3,1,female,Adult,Survived
4,3,male,Adult,Dead


In [28]:
# Categorizing Pclass
b = []
for i in titanic['Pclass']:
  if i == 1:
    b.append('First')
  if i == 2:
    b.append('Second')
  if i == 3:  
    b.append('Third')
titanic['Pclass_cat'] = b
titanic= titanic.drop('Pclass', axis=1)
titanic.head()

,Sex,Age_cat,Survived_Cat,Pclass_cat
0,male,Adult,Dead,Third
1,female,Adult,Survived,First
2,female,Adult,Survived,Third
3,female,Adult,Survived,First
4,male,Adult,Dead,Third


In [29]:
import pandas as pd
import csv
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [30]:
# Read in CSV file into an array of arrays
print(titanic.to_string(header=False))
titanic.to_csv('titanic.csv', index=False)
dataset = []
with open('titanic.csv') as f:
	reader = csv.reader(f)
	for row in reader:
		dataset.append(row)
for row in dataset: 
	print(row)


0      male  Adult      Dead   Third
1    female  Adult  Survived   First
2    female  Adult  Survived   Third
3    female  Adult  Survived   First
4      male  Adult      Dead   Third
5      male  Adult      Dead   Third
6      male  Adult      Dead   First
7      male  Child      Dead   Third
8    female  Adult  Survived   Third
9    female  Child  Survived  Second
10   female  Child  Survived   Third
11   female  Adult  Survived   First
12     male  Adult      Dead   Third
13     male  Adult      Dead   Third
14   female  Child      Dead   Third
15   female  Adult  Survived  Second
16     male  Child      Dead   Third
17     male  Adult  Survived  Second
18   female  Adult      Dead   Third
19   female  Adult  Survived   Third
20     male  Adult      Dead  Second
21     male  Adult  Survived  Second
22   female  Child  Survived   Third
23     male  Adult  Survived   First
24   female  Child      Dead   Third
25   female  Adult  Survived   Third
26     male  Adult      Dead   Third
2

In [31]:
# Transforming data for the apriori algorithm and finding support & confidence
oht = TransactionEncoder()
oht_ary = oht.fit(dataset).transform(dataset)
df = pd.DataFrame(oht_ary, columns=oht.columns_)
print(df)

frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
print(frequent_itemsets)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
print(rules[['antecedents', 'consequents', 'support', 'confidence']])


     Adult  Age_cat  Child   Dead  First  Pclass_cat  Second    Sex  Survived  \
0    False     True  False  False  False        True   False   True     False   
1     True    False  False   True  False       False   False  False     False   
2     True    False  False  False   True       False   False  False      True   
3     True    False  False  False  False       False   False  False      True   
4     True    False  False  False   True       False   False  False      True   
5     True    False  False   True  False       False   False  False     False   
6     True    False  False   True  False       False   False  False     False   
7     True    False  False   True   True       False   False  False     False   
8    False    False   True   True  False       False   False  False     False   
9     True    False  False  False  False       False   False  False      True   
10   False    False   True  False  False       False    True  False      True   
11   False    False   True  

In [32]:
#Transforming data for the apriori algorithm and finding support & confidence
oht = TransactionEncoder()
oht_ary = oht.fit(dataset).transform(dataset)
df = pd.DataFrame(oht_ary, columns=oht.columns_)
print(df)

frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
print(frequent_itemsets)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.85)
print(rules[['antecedents', 'consequents', 'support', 'confidence']])

     Adult  Age_cat  Child   Dead  First  Pclass_cat  Second    Sex  Survived  \
0    False     True  False  False  False        True   False   True     False   
1     True    False  False   True  False       False   False  False     False   
2     True    False  False  False   True       False   False  False      True   
3     True    False  False  False  False       False   False  False      True   
4     True    False  False  False   True       False   False  False      True   
5     True    False  False   True  False       False   False  False     False   
6     True    False  False   True  False       False   False  False     False   
7     True    False  False   True   True       False   False  False     False   
8    False    False   True   True  False       False   False  False     False   
9     True    False  False  False  False       False   False  False      True   
10   False    False   True  False  False       False    True  False      True   
11   False    False   True  